In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries
import os
import json
import glob
import random
import collections
import tqdm

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sample_sub = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
len(sample_sub)
##test data folder also had of 87 records 

In [ ]:
train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
train_df

In [ ]:
train_df.MGMT_value.value_counts()

In [ ]:
plt.figure(figsize=(5, 5))
sns.countplot(data=train_df, x="MGMT_value");
# plt.figure(figsize=(5, 5))
# sns.countplot(data=sample_sub, x="MGMT_value");

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data


def visualize_sample(
    brats21id,
    #the slice represents the way we slice the brain the first images represents a small slice 
    #the bigger slice_i the bigger the slice(front of the brain)
    slice_i,
    mgmt_value,
    types=("FLAIR", "T1w", "T1wCE", "T2w")
):
    #create a 16x16 figure with 5 inch space
    plt.figure(figsize=(16,5))
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/", 
        str(brats21id).zfill(5), 
        
    #the id in our csv file is 0 but 00000 in our directory that's why we use zfill
    )
    print(patient_path)
    for i, t in enumerate(types, 1):
        #a sorted list of each subject's images
        t_paths = sorted(
            #glob used to return all file paths that match a specific pattern.
            glob.glob(os.path.join(patient_path, t, "*")), 
            key=lambda x: int(x[:-4].split("-")[-1])
            
        )
        data = load_dicom(t_paths[int(len(t_paths) * slice_i)])
        #print(data.max())
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")

    plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.show()

In [ ]:
for i in range((train_df.shape[0])):
    print("***********************************************************")
    _brats21id = train_df.iloc[i]["BraTS21ID"]
    _mgmt_value = train_df.iloc[i]["MGMT_value"]
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5)
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.2)
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.3)

## Converting DCM files to niff

In [ ]:
pip install dicom2nifti

In [ ]:
import dicom2nifti

In [ ]:
patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/")

In [ ]:
os.makedirs('./output/train/00003/T2w')

In [ ]:
dicom2nifti.convert_directory("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00011/T2w", "output/train/00003/T2w")

In [ ]:
sample_filename = "./output/train/00003/T2w/5_t2w.nii.gz"

In [ ]:
sample_img = nib.load(sample_filename )
sample_img = np.asanyarray(sample_img.dataobj)
#print("img shape ->", sample_img.shape)
height, width, depth= sample_img.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")

In [ ]:
maxval = 409
i = np.random.randint(0, maxval)
print(f"Plotting Layer {i}")
plt.imshow(sample_img[:, i, :], cmap='gray')
plt.axis('off');

In [ ]:
os.makedirs('./test/T2w')

In [ ]:
import dicom2nifti
import os
dicom_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
out_folder = './test/T2w'
os.listdir(out_folder)
#for i in range(5):
#    dicom2nifti.convert_directory(dicom_directory, output_folder, compression=True, reorient=True)
  
directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/'
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename,'T2w')
    dicom2nifti.dicom_series_to_nifti(file_path, out_folder+'/'+filename+'_T2w.nii.gz')